대학, 학과, 교번, 성명, 풀네임, 약어, Scopus ID, Scopus ID 갯수, PNU ID, 직급, 이메일, 임용일자, 퇴직일자

In [77]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
from openpyxl import load_workbook

pd.options.display.float_format = '{:.0f}'.format

jeunyim_list = pd.read_excel("연구자명단/교무과제공_전임교원 명단_임용일자추가(2020.4.27.기준).xlsx")
giegum_list = pd.read_excel("연구자명단/교무과제공_기금교수 명단_임용일자추가(2020.4.20.현재기준).xlsx")
profess_list = pd.read_excel("연구자명단/도서관제공_교수급 연구자명단(원본).xlsx")
pip_all = pd.read_excel("PIP_ALL.xlsx")

total_prf_cnt = len(Researcher_list_v1)
jeunyim_cnt = len(jeunyim_list)
giegum_cnt = len(giegum_list)

Researcher_temp_list = df(columns=["대학","학과","성명","영문풀네임","영문약어","직급","교번","PNU_ID","SCOPUS_ID"])
Researcher_final_list = df(columns=["대학","학과","성명","영문풀네임","영문약어","직급","교번","PNU_ID","SCOPUS_ID"])
Researcher_list_v2 = df(columns=["대학","학과","성명","교번","풀네임","약어","S_ID",
                                 "S_ID개수","PNU_ID","직급","이메일","임용일","퇴직일"])

# 전임교원 명단 추가
for pro in range(0, jeunyim_cnt):
    res_number = jeunyim_list.loc[pro][5] #교번은 유일
    if res_number in profess_list['교번'].tolist():
        index_num = profess_list['교번'].tolist().index(res_number)
    else:
        index_num=40
    
    Researcher_temp_list.loc[pro] = [
        jeunyim_list.loc[pro][1], #대학
        jeunyim_list.loc[pro][2], #학과
        jeunyim_list.loc[pro][4], #성명
        profess_list.loc[index_num][4], #영문풀네임 from 도서관
        profess_list.loc[index_num][5], #영문약어 from 도서관
        jeunyim_list.loc[pro][3], #직급
        jeunyim_list.loc[pro][5], #교번
        profess_list.loc[index_num][0], #PNU_ID from 도서관
        "NONE"#jeunyim_list.loc[pro][0], #SCOPUS_ID
    ]
    
# 기금교원 명단 추가
for pro in range(0,giegum_cnt):
    res_number = giegum_list.loc[pro][4] #교번은 유일
    if res_number in profess_list['교번'].tolist():
        index_num = profess_list['교번'].tolist().index(res_number)
    else:
        index_num=40
    
    Researcher_temp_list.loc[jeunyim_cnt+pro] = [
        giegum_list.loc[pro][1], #대학
        giegum_list.loc[pro][2], #학과
        giegum_list.loc[pro][5], #성명
        profess_list.loc[index_num][4], #영문풀네임 from 도서관
        profess_list.loc[index_num][5], #영문약어 from 도서관
        giegum_list.loc[pro][3], #직급
        giegum_list.loc[pro][4], #교번
        profess_list.loc[index_num][0], #PNU_ID from 도서관
        "NONE"#jeunyim_list.loc[pro][0], #SCOPUS_ID
    ]
 
#print(Researcher_temp_list.shape)
#Researcher_temp_list
#Researcher_total_list.to_excel("researcher_table.xlsx")
# 학과명별로 재분류
pnu_department_list = Researcher_temp_list["학과"].unique()
tmp=[]
for department in pnu_department_list:
    tmp.append(Researcher_temp_list.groupby('학과').get_group(department).sort_values('성명', ascending=True))

for dep in range(0,len(tmp)):
    Researcher_final_list = pd.concat([Researcher_final_list,tmp[dep]])
    
#Researcher_final_list.to_excel("저자종합명단.xlsx")
Researcher_list_v1 = Researcher_final_list.reset_index(drop=True)

#Researcher_list_v1


In [79]:
for pro in range(0,total_prf_cnt):
    
    kyo_num = Researcher_list_v1.loc[pro][6]
    email=None
    app_date=None
    ret_date=None
    #print(kyo_num)
    if kyo_num in profess_list['교번'].tolist():
        email=profess_list['email'].loc[profess_list['교번'].tolist().index(kyo_num)]
    if kyo_num in pip_all['교직원번호'].tolist():
        app_date=pip_all['임용일자'].loc[pip_all['교직원번호'].tolist().index(kyo_num)]
        ret_date=pip_all['퇴직일자'].loc[pip_all['교직원번호'].tolist().index(kyo_num)]
    
    Researcher_list_v2.loc[pro] = [
        Researcher_list_v1.loc[pro][0], # 대학
        Researcher_list_v1.loc[pro][1], # 학과
        Researcher_list_v1.loc[pro][2], # 성명
        kyo_num, # 교번
        Researcher_list_v1.loc[pro][3], # 풀네임
        Researcher_list_v1.loc[pro][4], # 약어
        None,#Researcher_list_v1.loc[pro][], # Scopus id
        None,#Researcher_list_v1.loc[pro][], # Scopus id number
        Researcher_list_v1.loc[pro][7], # pnu id
        Researcher_list_v1.loc[pro][5], # 직급
        email, # 이메일
        app_date, # 임용일
        ret_date, # 퇴직일
    ]
                        
Researcher_list_v2.to_excel("저자종합명단.xlsx")

In [80]:
Researcher_list_v2

,대학,학과,성명,교번,풀네임,약어,S_ID,S_ID개수,PNU_ID,직급,이메일,임용일,퇴직일
0,공과대학,전기공학과,KANDASAMY PRABAKAR,111424,KANDASAMY PRABAKAR,"Prabakar, K",None,None,1100,교수,kprabakar@gmail.com,20080307,NaN
1,공과대학,전기공학과,권영안,110435,"Kwon, Young Ahn","Kwon, YA",None,None,1085,교수,yakwon@pusan.ac.kr,19870311,NaN
2,공과대학,전기공학과,김성신,110833,"Kim, Sungshin","Kim, S",None,None,1086,교수,sskim@pusan.ac.kr,19980325,NaN
3,공과대학,전기공학과,김욱,111573,"Kim, Wook","Kim, W",None,None,1087,부교수,NaN,20110901,NaN
4,공과대학,전기공학과,김장목,110901,"Kim, Jang-Mok","Kim, JM",None,None,1088,교수,jmok@pusan.ac.kr,20010301,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,부속연구소,점필재연구소,신상필,111803,NaN,NaN,None,None,895,조교수,None,20170831,NaN
1341,부속연구소,점필재연구소,임상석,111958,NaN,NaN,None,None,895,조교수,None,20190801,NaN
1342,치의학전문대학원,치의생명과학과,김형식,111867,NaN,NaN,None,None,895,조교수,None,20180901,NaN
1343,공과대학,고분자공학과,김채빈,111919,NaN,NaN,None,None,895,조교수,None,20190920,NaN


In [27]:
pip_all.loc[0]

성명                                               하만영
대학                                              공과대학
학과                                             기계공학부
전공                                          에너지시스템전공
직급                                                교수
논문종류                                   국내학회 논문(전국학회)
제목       두 실린더의 수직 위치가 수평 채널 내부의 3차원 자연대류 현상에 미치는 영향
게재년월                                         2018/02
게재지명                                     대한기계학회 논문집B
논문구분                                      SCOPUS 등재 
학술지색인                                         SCOPUS
국내외구분                                             국내
임용일자                                     1.99103e+07
퇴직일자                                             NaN
교직원번호                                         110530
Name: 0, dtype: object

In [33]:
profess_list['email'].loc[0]

'nursing@pusan.ac.kr'

In [39]:
profess_list['교번'].tolist().index(110998)

0

In [37]:
profess_list.loc[0]

ID                            840
교번                         110998
메모                            NaN
연구자명                          강인순
영문풀네임1              Kang, In-Soon
영문약어1                    Kang, IS
신분                             교수
단대                           간호대학
학과                           간호학과
전공                            NaN
email         nursing@pusan.ac.kr
ScopusID 1                    NaN
ScopusID 2                    NaN
ScopusID 3                    NaN
Name: 0, dtype: object

In [40]:
profess_list['email'].loc[profess_list['교번'].tolist().index(110998)]

'nursing@pusan.ac.kr'

In [63]:
profess_list['교번'].tolist()

[110998,
 111375,
 110719,
 110647,
 640477,
 110718,
 111085,
 111601,
 110855,
 110780,
 110914,
 111374,
 111308,
 111511,
 111167,
 111509,
 111275,
 111512,
 110850,
 110866,
 111607,
 110629,
 111311,
 110990,
 111446,
 640442,
 110449,
 110371,
 110991,
 111019,
 111480,
 110989,
 111587,
 111773,
 111667,
 111676,
 111716,
 111068,
 110440,
 110333,
 110828,
 111388,
 110834,
 111067,
 111274,
 111485,
 110448,
 110457,
 111312,
 111564,
 110421,
 110630,
 110406,
 111137,
 110521,
 110539,
 411267,
 110246,
 110432,
 111408,
 111225,
 111150,
 110655,
 111416,
 110504,
 111009,
 111326,
 111149,
 110887,
 111223,
 111728,
 110953,
 640380,
 110806,
 640116,
 110473,
 111650,
 111570,
 640425,
 110474,
 640414,
 640426,
 110369,
 110338,
 640465,
 110453,
 640368,
 111266,
 111039,
 111272,
 111656,
 640391,
 111271,
 110176,
 111267,
 111338,
 110212,
 111270,
 110178,
 111489,
 110048,
 111437,
 110181,
 110511,
 110568,
 110045,
 110207,
 110483,
 111630,
 110865,
 111220,
 